In [3]:
import pennylane as qml
import pandas as pd
import qutip
from pennylane import numpy as np

#Inputs
n_layers = 2
n_wires = 4
iteracoes = 4
n_targets = 10


def BasicEntanglerLayers(n_layers, n_wires, n_targets):

    shape = qml.BasicEntanglerLayers.shape(n_layers, n_wires)
    weights = np.random.random(size=shape)

    print("\nVetor de matrizes de peso: ", weights)


    #Cria dados
    def normaliza(v):
        v = v / (np.linalg.norm(v))    
        return v

    def cria_alvos(n_targets, n_wires):
        target_states = []
        for i in range(n_targets):
            target_states.append(normaliza(np.random.rand(1, 2**(n_wires))))
        return target_states


    @qml.qnode(dev)
    def circuit(weights):
        qml.BasicEntanglerLayers(weights=weights, wires=range(n_wires))
        return qml.state()
    

    def ket_in_amplt(v):
        aux = []
        for i in range(len(v)):
            v[i] = np.dot(v[i],v[i])
            v[i] = np.linalg.norm(v[i])
            aux.append(float(v[i].real))
        np.array(aux)    
        return aux
    

    print("\n")
    print(qml.draw(circuit, expansion_strategy="device")(weights))
    ket = circuit(weights)
    amplt = ket_in_amplt(ket)
    print("\nEstado resultado do circuito: ", ket)
    target_states = cria_alvos(n_targets, n_wires)



    def fidelity(target_states, amplt):
        fidelity_sum = 0
        for target_state in target_states:
            output_probs = amplt
            fidelity = np.vdot(target_state, output_probs) ** 2
            fidelity_sum += fidelity
    
        average_fidelity = fidelity_sum / len(target_states)
        return average_fidelity



    def compute_Q_ptrace(ket, N):
        ket = qutip.Qobj(ket, dims=[[2]*(N), [1]*(N)]).unit()
        entanglement_sum = 0
        for k in range(N):
            rho_k_sq = ket.ptrace([k])**2
            entanglement_sum += rho_k_sq.tr()  
   
        Q = 2*(1 - (1/N)*entanglement_sum)
        return Q



    Q = compute_Q_ptrace(ket, n_wires)
    #print("\nGrau de Emaranhamento: {}".format(Q))

    E = fidelity(target_states, amplt)
    #print("Expressabilidade: {}".format(E))

    #print("\n\nEstado gerado: {}".format(ket))
    #print("\n\n\n\n\n")

    k=[Q,E]


    return k


dev = qml.device('default.qubit', wires=n_wires)


k=[]
for i in range(iteracoes):
    print("\n=============")
    print("= Modelo: {} =".format(i+1))
    print("=============\n")
    k.append(BasicEntanglerLayers(n_layers, n_wires, n_targets))


print("\n\nResultados:\n")
for i in range(iteracoes):
    print(" Modelo {} \n Grau de emaranhamento: {} \n Expressabilidade: {} \n".format(i+1,k[i][0],k[i][1]))


def pand(k):
    eman=[]
    exp=[]

    for i in range(iteracoes):
        eman.append(k[i][0])
    for i in range(iteracoes):
        exp.append(k[i][1]) 

    dic = {"Grau de Emaranhamento": eman, "Expressabilidade":exp}
    df = pd.DataFrame(dic)
    return df

df = pand(k)
df.describe()


= Modelo: 1 =


Vetor de matrizes de peso:  [[6.51870577e-04 1.28099063e-02 9.21806124e-01 5.14103460e-01]
 [8.70536015e-01 6.45649330e-01 1.30178804e-02 3.30418297e-01]]


0: ──RX(0.00)─╭●───────╭X──RX(0.87)─╭●───────╭X─┤  State
1: ──RX(0.01)─╰X─╭●────│───RX(0.65)─╰X─╭●────│──┤  State
2: ──RX(0.92)────╰X─╭●─│───RX(0.01)────╰X─╭●─│──┤  State
3: ──RX(0.51)───────╰X─╰●──RX(0.33)───────╰X─╰●─┤  State

Estado resultado do circuito:  [0.54034132+0.j 0.00457447+0.j 0.02935697+0.j 0.00067048+0.j
 0.00109696+0.j 0.13285373+0.j 0.00255205+0.j 0.11744536+0.j
 0.01317338+0.j 0.02278068+0.j 0.0149139 +0.j 0.00972525+0.j
 0.00598343+0.j 0.00325864+0.j 0.04083503+0.j 0.06043836+0.j]

= Modelo: 2 =


Vetor de matrizes de peso:  [[0.30601268 0.56034497 0.74413701 0.35660758]
 [0.87057319 0.1782717  0.57926347 0.59230007]]


0: ──RX(0.31)─╭●───────╭X──RX(0.87)─╭●───────╭X─┤  State
1: ──RX(0.56)─╰X─╭●────│───RX(0.18)─╰X─╭●────│──┤  State
2: ──RX(0.74)────╰X─╭●─│───RX(0.58)────╰X─╭●─│──┤  State
3: ──RX(

,Grau de Emaranhamento,Expressabilidade
count,4.000000,4.000000
mean,0.186438,0.054670
std,0.102079,0.005137
min,0.054099,0.047202
25%,0.132303,0.053373
50%,0.209081,0.056526
75%,0.263217,0.057823
max,0.273491,0.058424
